<a href="https://colab.research.google.com/github/LeRoiBof/ML-Project/blob/main/Copie_de_ML1_Project_T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - Machine Learning - Gradient Boosting Regression with XGBoost

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

## Data loading


In [ ]:
df_train = pd.read_csv('train.csv')
y_column = 'LotArea'
X_train, y_train = df_train.drop(y_column, axis=1), df_train[y_column]

X_test = pd.read_csv('test.csv')
X_test_id = X_test['ID']
X_test = X_test.drop(columns='ID')

X_train

,MSSubClass,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,120,RM,NaN,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,NaN,0,6,2008,ConLD,Normal
1,20,RL,66.0,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
2,60,RL,74.0,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,NaN,0,10,2006,WD,Normal
3,90,RL,70.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,NaN,0,12,2007,WD,Normal
4,30,RM,50.0,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,...,0,0,NaN,MnPrv,NaN,0,3,2007,WD,Alloca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,60,RL,82.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,180,0,NaN,NaN,NaN,0,7,2009,WD,Normal
1455,20,RL,60.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
1456,160,RL,24.0,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,NaN,NaN,NaN,0,9,2009,WD,Normal
1457,20,RL,61.0,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,GdPrv,NaN,0,5,2007,WD,Normal


## Preprocessing

In [ ]:
from re import X


selected_features = ['MSSubClass','MSZoning','LotFrontage','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','Heating','HeatingQC','CentralAir','Electrical','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond','PavedDrive','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','PoolQC','Fence','MiscFeature','MiscVal','MoSold','YrSold','SaleType','SaleCondition']
X_train = X_train[selected_features]
X_test = X_test[selected_features]

numeric_features = ['MSSubClass', 'LotFrontage','OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']
categorical_features = ['MSZoning', 'Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','OverallQual','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = Pipeline(steps=[('preprocessor', preprocessor), ('model', XGBRegressor())])

## Fitting and prediction

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


## Submission

In [ ]:
submission = pd.DataFrame({
    'ID': X_test_id,
    'LotArea': y_pred
})
submission.to_csv('submission.csv', index=False)
submission

,ID,LotArea
0,0,6013.540527
1,1,10492.340820
2,2,9813.858398
3,3,16375.561523
4,4,11921.013672
...,...,...
1455,1455,7758.960938
1456,1456,6475.497559
1457,1457,12163.889648
1458,1458,7978.661621
